In [1]:
# %%
# imports
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import itertools
from prophet import Prophet
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error, mean_absolute_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np
import plotly.express as px
from pmdarima.preprocessing import FourierFeaturizer
from pmdarima import auto_arima, ARIMA
import matplotlib.pyplot as plt
import holidays
pd.options.plotting.backend = 'plotly'

In [2]:
df_store = pd.read_pickle('data/df_daily.pkl')
df_company = df_store.groupby('date').sum()
train_data = df_company['sales']/1e6
train_data.index.freq = 'D'

In [3]:
train_data.plot(title='Sales TS in Million VND (aggregated data)')

In [4]:
# yearly seasonality
fouri_terms = FourierFeaturizer(365.25, 2)
y_prime, df_fouri = fouri_terms.fit_transform(train_data)
df_fouri.index = y_prime.index


In [5]:
df_fouri.head()

,FOURIER_S365-0,FOURIER_C365-0,FOURIER_S365-1,FOURIER_C365-1
date,,,,
2017-08-07,0.017202,0.999852,0.034398,0.999408
2017-08-08,0.034398,0.999408,0.068755,0.997634
2017-08-09,0.051584,0.998669,0.103031,0.994678
2017-08-10,0.068755,0.997634,0.137185,0.990545
2017-08-11,0.085906,0.996303,0.171177,0.985240


In [6]:
# holiday
ts_holiday = pd.read_pickle('data/holiday.pkl')
print(ts_holiday.index)

DatetimeIndex(['2018-02-15', '2018-02-16', '2018-02-17', '2018-02-18',
               '2018-02-19', '2018-02-20', '2018-05-01', '2019-02-04',
               '2019-02-05', '2019-02-06', '2019-02-07', '2019-02-08',
               '2019-02-09', '2019-05-01', '2020-01-24', '2020-01-25',
               '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
               '2020-05-01'],
              dtype='datetime64[ns]', name='date', freq=None)


In [ ]:
# promo => causing DATA LEAK, will not be used
#ts_promo = df_company['promo_count']

## ARIMA model

In [7]:
# combine exog. variables
df_exog = pd.concat([df_fouri, ts_holiday], axis=1)
df_exog['holiday'] = df_exog['holiday'].fillna(False).astype('int')

In [8]:
df_exog.head()

,FOURIER_S365-0,FOURIER_C365-0,FOURIER_S365-1,FOURIER_C365-1,holiday
date,,,,,
2017-08-07,0.017202,0.999852,0.034398,0.999408,0
2017-08-08,0.034398,0.999408,0.068755,0.997634,0
2017-08-09,0.051584,0.998669,0.103031,0.994678,0
2017-08-10,0.068755,0.997634,0.137185,0.990545,0
2017-08-11,0.085906,0.996303,0.171177,0.985240,0


In [9]:
# Split the time series as well as exogenous features data into train and test splits
steps_ahead = 14 # 2 weeks
y_to_train = y_prime.iloc[:-steps_ahead]
y_to_test = y_prime.iloc[-steps_ahead:]

exog_to_train = df_exog.iloc[:-steps_ahead]
exog_to_test = df_exog.iloc[-steps_ahead:]

In [10]:
# Fit model to the level to find common order
arima_model = auto_arima(
    y=y_to_train,
    exogenous=exog_to_train,
    D=1, 
    seasonal=True, m=7 # Weekly seasonality
    )
arima_model

ARIMA(order=(4, 0, 0), scoring_args={}, seasonal_order=(2, 1, 0, 7),
      suppress_warnings=True, with_intercept=False)

In [11]:
# Forecast
arima_y_fitted = arima_model.predict_in_sample(
    X=exog_to_train)
arima_y_forecast = arima_model.predict(
    n_periods=len(y_to_test), exogenous=exog_to_test)
arima_y_forecast = pd.Series(arima_y_forecast,
                             name='forecast',
                             index=y_to_test.index)

In [12]:
# metrics
# in-sample
mae_IS = round(mean_absolute_error(y_to_train, arima_y_fitted))
mape_IS = round(mean_absolute_percentage_error(
    y_to_train, arima_y_fitted), 3)

# out-sample
mae_OOS = round(mean_absolute_error(y_to_test, arima_y_forecast))
mape_OOS = round(mean_absolute_percentage_error(
    y_to_test, arima_y_forecast), 3)

print(f'Company:')
print('Out of sample:',
      f'MAPE: {mape_OOS}',
      f'MAE: {mae_OOS}\n',
      sep='\n')

Company:
Out of sample:
MAPE: 0.155
MAE: 140



In [13]:
print(arima_model)
res_whole_holiday_promo = pd.DataFrame()

 ARIMA(4,0,0)(2,1,0)[7]          


In [14]:
# LOOP all stores
for store in df_store['store_id'].unique():  # print(store)
    df_data = df_store[df_store['store_id'] == store].set_index('date')[
        ['sales', 'promo_count']]
    df_data.index.freq = 'D'
    train_data = df_data['sales']/1e6

    # Prepare the fourier terms to model annual seasonality; add as exogenous features to auto_arima
    fouri_terms = FourierFeaturizer(365.25, 2)
    y_prime, df_fouri = fouri_terms.fit_transform(train_data)
    # is exactly the same as manual calculation in the above cells
    df_fouri.index = y_prime.index

    # holiday
    # ts_holiday = pd.read_pickle('data/holiday.pkl')

    # promo
    #ts_promo = df_data['promo_count']

    # combine exog. variables
    df_exog = pd.concat([df_fouri, ts_holiday], axis=1)
    df_exog['holiday'] = df_exog['holiday'].fillna(False).astype('int')
    df_exog.dropna(inplace=True)

    # Split the time series as well as exogenous features data into train and test splits
    steps_ahead = 14
    y_to_train = y_prime.iloc[:-steps_ahead]
    y_to_test = y_prime.iloc[-steps_ahead:]

    exog_to_train = df_exog.iloc[:-steps_ahead]
    exog_to_test = df_exog.iloc[-steps_ahead:]

    # Fit model to each store in the level
    arima_model_store = ARIMA(
        order=arima_model.order,
        seasonal_order=arima_model.seasonal_order
    ).fit(y_to_train,
          X=exog_to_train
          )

    # Forecast
    arima_y_fitted = arima_model_store.predict_in_sample(
        X=exog_to_train)
    arima_y_forecast = arima_model_store.predict(
        n_periods=len(y_to_test), exogenous=exog_to_test)
    arima_y_forecast = pd.Series(arima_y_forecast,
                                 name='forecast',
                                 index=y_to_test.index)
    # Random walk
    RW_y_forecast = pd.Series(
        y_to_train[-1], name='RW_forecast', index=y_to_test.index)

    # metrics
    # in-sample
    mae_IS = round(mean_absolute_error(y_to_train, arima_y_fitted))
    mape_IS = round(mean_absolute_percentage_error(
        y_to_train, arima_y_fitted), 3)

    # out-sample
    mae_OOS = round(mean_absolute_error(y_to_test, arima_y_forecast))
    mape_OOS = round(mean_absolute_percentage_error(
        y_to_test, arima_y_forecast), 3)
    RW_mae_OOS = round(mean_absolute_error(y_to_test, RW_y_forecast))
    RW_mape_OOS = round(mean_absolute_percentage_error(
        y_to_test, RW_y_forecast), 3)

    print(f'Store {store}')
    print(f'MAPE: {mape_OOS}',
          f'MAE: {mae_OOS}\n',
          sep='\n')

    res_whole_holiday_promo = res_whole_holiday_promo.append({
        'store_id': store,
        'fc_IS': arima_y_fitted,
        'fc_OOS': arima_y_forecast,
        'fc_RW': RW_y_forecast,
        'arima_mape_OOS': mape_OOS,
        'arima_mape_IS': mape_IS,
        'arima_mae_OOS': mae_OOS,
        'arima_mae_IS': mae_IS,
        'RW_mape_OOS': RW_mape_OOS,
        'RW_mae_OOS': RW_mae_OOS,
    }, ignore_index=True)

Store 307222
MAPE: 0.537
MAE: 15

Store 307244
MAPE: 0.319
MAE: 6

Store 307248
MAPE: 0.298
MAE: 7

Store 320264
MAPE: 0.566
MAE: 11

Store 328165
MAPE: 0.253
MAE: 16

Store 349920
MAPE: 0.256
MAE: 10

Store 349924
MAPE: 0.327
MAE: 8

Store 349952
MAPE: 0.288
MAE: 8

Store 349958
MAPE: 0.351
MAE: 11

Store 349962
MAPE: 0.348
MAE: 5

Store 349972
MAPE: 0.31
MAE: 10

Store 349978
MAPE: 0.388
MAE: 4

Store 349980
MAPE: 0.389
MAE: 13

Store 349998
MAPE: 0.316
MAE: 6

Store 350016
MAPE: 0.268
MAE: 7



/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Store 350018
MAPE: 0.19
MAE: 12

Store 350026
MAPE: 0.52
MAE: 6

Store 350028
MAPE: 0.36
MAE: 4

Store 350040
MAPE: 0.491
MAE: 8

Store 350046
MAPE: 0.222
MAE: 3

Store 350054
MAPE: 0.279
MAE: 8

Store 350056
MAPE: 0.382
MAE: 4

Store 350060
MAPE: 0.201
MAE: 6

Store 354468
MAPE: 0.521
MAE: 4

Store 387240
MAPE: 0.399
MAE: 4

Store 412585
MAPE: 0.374
MAE: 4



/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



Store 441997
MAPE: 0.329
MAE: 4



...

Store 536902
MAPE: 0.34
MAE: 4



/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



Store 566790
MAPE: 0.428
MAE: 12



/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



Store 566792
MAPE: 0.56
MAE: 4



In [38]:
res_whole_holiday_promo.to_csv('results/res_RW_ARIMA_whole_holiday_promo.csv')
res_whole_holiday_promo = pd.read_csv('results/res_RW_ARIMA_whole_holiday_promo.csv', index_col=0)

In [34]:
res_whole_holiday_promo.shape

(38, 10)

## FB Prophet

In [ ]:
fb_df = pd.concat([df_company[['sales']], ts_holiday], axis=1)
fb_df['sales'] = fb_df['sales']/1e6
fb_df['holiday'] = fb_df['holiday'].fillna(False).astype('bool')
fb_df = fb_df.reset_index().rename({'date': 'ds', 'sales': 'y'}, axis=1)
fb_train = fb_df.iloc[:-steps_ahead]
fb_test = fb_df.iloc[-steps_ahead:]

In [ ]:
# Grid search set up
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

all_params = [dict(zip(param_grid.keys(), v))
              for v in itertools.product(*param_grid.values())]

In [ ]:
mape_PROPHET = []
for params in all_params:
    # set up model
    m = Prophet(**params)
    m.add_regressor('holiday')
    m.fit(fb_train)
    # set up CV
    df_cv = cross_validation(
        m,
        initial=(str(fb_train.shape[0]-steps_ahead*4)+' days'),
        period='7 days', horizon='14 days')
    # evaluate
    df_p = performance_metrics(df_cv)
    mape_PROPHET.append(df_p['mape'].values[-1])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mape_PROPHET
tuning_results.sort_values('mape', inplace=True)

In [ ]:
# %% final PROPHET
fb_model = Prophet(
    changepoint_prior_scale=tuning_results.loc[0, 'changepoint_prior_scale'],
    seasonality_prior_scale=tuning_results.loc[0, 'seasonality_prior_scale'],
)

fb_model.fit(fb_df)
fb_fc = fb_model.predict(fb_test)
mape_PROPHET = round(mean_absolute_percentage_error(fb_test['y'], fb_fc['yhat']), 3)
mape_PROPHET

In [ ]:
res_fb = pd.DataFrame()
# LOOP all stores
for store in df_store['store_id'].unique():  # print(store)
    df_data = df_store[df_store['store_id'] == store].set_index('date')[
        ['sales']]
    fb_df = pd.concat([df_data, ts_holiday], axis=1)
    fb_df['sales'] = fb_df['sales']/1e6
    fb_df['holiday'] = fb_df['holiday'].fillna(False).astype('bool')
    fb_df = fb_df.dropna().reset_index().rename(
        {'date': 'ds', 'sales': 'y'}, axis=1)
    fb_train = fb_df.iloc[:-steps_ahead]
    fb_test = fb_df.iloc[-steps_ahead:]

    fb_model = Prophet(
        changepoint_prior_scale=tuning_results.loc[0,'changepoint_prior_scale'],
        seasonality_prior_scale=tuning_results.loc[0,'seasonality_prior_scale'],
    )
    fb_model.add_regressor('holiday')
    #fb_model.add_regressor('promo_count')
    fb_model.fit(fb_train)
    fb_fc = fb_model.predict(fb_test)

    # metrics
    fb_mae_OOS = round(mean_absolute_error(fb_test['y'], fb_fc['yhat']))
    fb_mape_OOS = round(mean_absolute_percentage_error(
        fb_test['y'], fb_fc['yhat']), 3)

    print(f'Store {store}')
    print(f'MAPE: {fb_mape_OOS}',
          f'MAE: {fb_mae_OOS}\n',
          sep='\n')

    res_fb = res_fb.append({
        'store_id': store,
        'fb_fc': fb_fc['yhat'],
        'fb_mape_OOS': fb_mape_OOS,
        'fb_mae_OOS': fb_mae_OOS,
    }, ignore_index=True)

In [ ]:
# Save results to csv
res_fb.to_csv('results/res_fb_noPromo.csv')
res_fb = pd.read_csv('results/res_fb_noPromo.csv')

In [ ]:
res = pd.merge(res_fb.set_index('store_id')['fb_mape_OOS'],
               res_whole_holiday_promo.set_index(
                   'store_id')[['RW_mape_OOS', 'arima_mape_OOS']],
               'inner', left_index=True, right_index=True).reset_index()

# %%
fig = res[['fb_mape_OOS',
        #    'RW_mape_OOS',
           'arima_mape_OOS'
           ]].plot()

# RW_mean = round(res['RW_mape_OOS'].mean(), 3)
# fig.add_hline(y=RW_mean, line_dash="dot", line_color='red',
#               annotation_text=str(RW_mean))

arima_mean = round(res['arima_mape_OOS'].mean(), 3)
fig.add_hline(y=arima_mean, line_dash="dot", line_color='red',
              annotation_text=str(arima_mean)
        )

fb_mean = round(res['fb_mape_OOS'].mean(), 3)
fig.add_hline(y=fb_mean, line_dash="dot", line_color='blue',
              annotation_text=str(fb_mean),
              annotation_position="bottom right")

## FB Prophet with sales

In [51]:
fb_df = pd.concat([df_company[['sales']], ts_holiday], axis=1)
fb_df['sales'] = fb_df['sales']/1e6
#fb_df['holiday'] = fb_df['holiday'].fillna(False).astype('bool')
fb_df = fb_df.reset_index().rename({'date': 'ds', 'sales': 'y'}, axis=1)
fb_train = fb_df.iloc[:-steps_ahead]
fb_test = fb_df.iloc[-steps_ahead:]

In [53]:
import datetime

holiday = pd.DataFrame(holidays.Vietnam(years=[2018, 2019, 2020, 2021]).items()).rename({0:'date', 1:'holiday_neg'}, axis=1)
# Add Tet promotion
promo = pd.DataFrame(holiday[holiday.holiday_neg == 'Vietnamese New Year'].date - datetime.timedelta(days=1))
promo['promotion'] = "1 days before Tet Holiday"
# Add Black Friday
promo = promo.append(
            pd.DataFrame({'date':[datetime.date(2020,11,27), datetime.date(2019,11,29),datetime.date(2018,11,23)],
                          'promotion':["Black Friday","Black Friday","Black Friday"]})
        )

#holiday['promotion'] = np.NaN
# All together
#holiday = holiday.append(promo).sort_values('date')

# set to 1 if holiday affect sales negatively
holiday_off = holiday.replace({'Vietnamese New Year.*': 1, 
                 '.*day of Tet Holiday': 1,
                 'International Labor Day': 1,
                 '\D': np.NaN}, 
                regex=True).dropna()

promotions = pd.DataFrame({
        'holiday':'big_promotion',
        'ds':promo.date,
        'lower_window': -14, # 2 weeks before
        'upper_window': 0,
})

holiday_negative = pd.DataFrame({
        'holiday':'big_promotion',
        'ds':holiday_off.date,
        'lower_window': 0,
        'upper_window': 0,
})

holidays_fb = pd.concat((promotions, holiday_negative))

In [45]:
# Grid search set up
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

all_params = [dict(zip(param_grid.keys(), v))
              for v in itertools.product(*param_grid.values())]

In [46]:
holidays_fb.head()

,holiday,ds,lower_window,upper_window
2,big_promotion,2018-02-15,-14,0
13,big_promotion,2019-02-04,-14,0
24,big_promotion,2020-01-24,-14,0
35,big_promotion,2021-02-11,-14,0
0,big_promotion,2020-11-27,-14,0


In [54]:
mape_PROPHET = []
for params in all_params:
    params['weekly_seasonality']=True
    params['holidays'] = holidays_fb
    # set up model
    m = Prophet(**params)
    #m.add_regressor('holiday')
    m.fit(fb_train)
    # set up CV
    df_cv = cross_validation(
        m,
        initial=(str(fb_train.shape[0]-steps_ahead*4)+' days'),
        period='7 days', horizon='14 days')
    # evaluate
    df_p = performance_metrics(df_cv)
    mape_PROPHET.append(df_p['mape'].values[-1])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mape_PROPHET
tuning_results.sort_values('mape', inplace=True)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 6 forecasts with cutoffs between 2020-11-29 00:00:00 and 2021-01-03 00:00:00


Initial log joint probability = -12.6088
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2690.23   0.000942487       4762.13       1.494      0.1494      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     171       2707.42   3.97871e-09       4557.34      0.3216      0.3216      210   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


  0%|          | 0/6 [00:00<?, ?it/s]

Initial log joint probability = -11.5148
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      83       2561.27   7.95479e-09       4885.81      0.4348      0.4348      110   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


 17%|█▋        | 1/6 [00:02<00:12,  2.40s/it]

...

 67%|██████▋   | 4/6 [00:11<00:05,  2.89s/it]

Initial log joint probability = -10.3383
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2836.77    0.00136023       30.1981           1           1      114   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2838.35   5.75009e-05       29.7639      0.3864      0.3864      222   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       2838.93    0.00111709       24.8602           1           1      346   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     397       2839.28   7.70968e-05       20.6299   8.455e-06       0.001      518  LS failed, Hessian reset 
     399       2839.28   0.000109152       13.2148           1           1      520   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       2839.42    0.00666707    

 83%|████████▎ | 5/6 [00:14<00:02,  2.92s/it]

Initial log joint probability = -8.9084
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2853.45    0.00849618       65.6729           1           1      122   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2854.94    0.00284011       21.5401           1           1      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        2855.4   0.000816089       19.2178           1           1      359   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399        2855.8   0.000323586       19.7324      0.8717      0.8717      475   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       2856.06     0.0254962       37.5784           1           1      591   
    Iter      log prob        ||dx||      ||grad||       alpha   

100%|██████████| 6/6 [00:17<00:00,  2.85s/it]


In [55]:
tuning_results

,changepoint_prior_scale,seasonality_prior_scale,weekly_seasonality,holidays,mape
3,0.001,10.00,True,holiday ds lower_window up...,0.119782
2,0.001,1.00,True,holiday ds lower_window up...,0.125804
0,0.001,0.01,True,holiday ds lower_window up...,0.130599
1,0.001,0.10,True,holiday ds lower_window up...,0.137020
12,0.500,0.01,True,holiday ds lower_window up...,0.209644
10,0.100,1.00,True,holiday ds lower_window up...,0.211387
9,0.100,0.10,True,holiday ds lower_window up...,0.212392
11,0.100,10.00,True,holiday ds lower_window up...,0.212490
8,0.100,0.01,True,holiday ds lower_window up...,0.225696
15,0.500,10.00,True,holiday ds lower_window up...,0.240903


In [56]:
# %% final PROPHET
params = {
    'changepoint_prior_scale':tuning_results.loc[0, 'changepoint_prior_scale'],
    'seasonality_prior_scale':tuning_results.loc[0, 'seasonality_prior_scale'],
    'weekly_seasonality':True,
    'holidays':holidays_fb
}

fb_model = Prophet(**params)

fb_model.fit(fb_df)
fb_fc = fb_model.predict(fb_test)
mape_PROPHET = round(mean_absolute_percentage_error(fb_test['y'], fb_fc['yhat']), 3)
mape_PROPHET

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -16.0885
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2725.42   1.12611e-08       4764.26      0.5451      0.5451      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     100       2725.42   9.18273e-09       4539.92      0.3972           1      133   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


0.198

In [57]:
# LOOP all stores
res_fb = pd.DataFrame()
for store in df_store['store_id'].unique():  # print(store)
    df_data = df_store[df_store['store_id'] == store].set_index('date')[
        ['sales']]
    fb_df = pd.concat([df_data], axis=1)
    fb_df['sales'] = fb_df['sales']/1e6
    fb_df = fb_df.dropna().reset_index().rename(
        {'date': 'ds', 'sales': 'y'}, axis=1)
    fb_train = fb_df.iloc[:-steps_ahead]
    fb_test = fb_df.iloc[-steps_ahead:]

    params = {
    'changepoint_prior_scale':tuning_results.loc[0, 'changepoint_prior_scale'],
    'seasonality_prior_scale':tuning_results.loc[0, 'seasonality_prior_scale'],
    'weekly_seasonality':True,
    'holidays':holidays_fb
    }

    fb_model = Prophet(**params)
    #fb_model.add_regressor('promo_count')
    fb_model.fit(fb_train)
    fb_fc = fb_model.predict(fb_test)

    # metrics
    fb_mae_OOS = round(mean_absolute_error(fb_test['y'], fb_fc['yhat']))
    fb_mape_OOS = round(mean_absolute_percentage_error(
        fb_test['y'], fb_fc['yhat']), 3)

    print(f'Store {store}')
    print(f'MAPE: {fb_mape_OOS}',
          f'MAE: {fb_mae_OOS}\n',
          sep='\n')

    res_fb = res_fb.append({
        'store_id': store,
        'fb_fc': fb_fc['yhat'],
        'fb_mape_OOS': fb_mape_OOS,
        'fb_mae_OOS': fb_mae_OOS,
    }, ignore_index=True)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -7.57649
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2715.72   2.02769e-08       5014.36      0.3725      0.9193      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     103       2715.72   5.37251e-09       5088.38      0.2814           1      130   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 307222
MAPE: 0.237
MAE: 8

Initial log joint probability = -8.63274
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      77       2556.19   2.55704e-09       4994.53      0.2362      0.2362      102   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 307248
MAPE: 0.167
MAE: 5

Initial log joint probability = -4.99374
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      87       2967.16   7.68754e-09       4546.49      0.3437      0.3437      117   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store 320264
MAPE: 0.343
MAE: 7

Initial log joint probability = -20.2113
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      63       2581.64   4.36665e-05       5525.74    8.92e-09       0.001      124  LS failed, Hessian reset 
      95       2582.03   2.35163e-05        4663.7   4.737e-09       0.001      204  LS failed, Hessian reset 
      99       2582.09   1.02314e-05        5204.8      0.2078           1      210   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     126       2582.16   1.23947e-06       4982.86   2.427e-10       0.001      282  LS failed, Hessian reset 
     166       2582.18   8.37444e-09       4509.18      0.5592      0.5592      333   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


KeyboardInterrupt: 

In [ ]:
# Save results to csv
res_fb.to_csv('results/res_fb_withPromo.csv')


## Results

In [39]:

res_ARIMA = pd.read_csv('results/res_RW_ARIMA_whole_holiday_promo.csv',index_col=0)
res_fb_noPromo = pd.read_csv('results/res_fb_noPromo.csv',index_col=0)
res_fb_withPromo = pd.read_csv('results/res_fb_withPromo.csv',index_col=0)

In [40]:
res_ARIMA.shape
# res_ARIMA.set_index('store_id').sort_index()

(38, 10)

In [41]:
res = pd.merge(
    res_fb_noPromo.set_index('store_id')['fb_mape_OOS'],
    res_ARIMA.set_index('store_id')[['RW_mape_OOS', 'arima_mape_OOS']],
    'inner', left_index=True, right_index=True).reset_index()
res.head()

,store_id,fb_mape_OOS,RW_mape_OOS,arima_mape_OOS
0,307222.0,0.291,0.322,0.537
1,307244.0,0.515,0.555,0.319
2,307248.0,0.192,0.334,0.298
3,320264.0,0.366,0.497,0.566
4,328165.0,0.315,0.900,0.253


In [48]:

res = pd.merge(
    res_fb_noPromo.set_index('store_id')['fb_mape_OOS'],
    res_ARIMA.set_index('store_id')[['RW_mape_OOS', 'arima_mape_OOS']],
    'inner', left_index=True, right_index=True)
res = pd.merge(
    res,
    res_fb_withPromo.set_index('store_id').rename(
        {'fb_mape_OOS': 'fb_withPromo_mape_OOS'}, axis=1)['fb_withPromo_mape_OOS'],
    'inner', left_index=True, right_index=True).reset_index()


# %%
fig = res[['fb_mape_OOS',
           'fb_withPromo_mape_OOS',
        #    'RW_mape_OOS',
           'arima_mape_OOS'
           ]].plot()

# RW_mean = round(res['RW_mape_OOS'].mean(), 3)
# fig.add_hline(y=RW_mean, line_dash="dot", line_color='red',
#               annotation_text=str(RW_mean),
#               annotation_position="top right"
#               )

arima_mean = round(res['arima_mape_OOS'].mean(), 3)
fig.add_hline(y=arima_mean, line_dash="dot", line_color='green',
              annotation_text=str(arima_mean),
              annotation_position="bottom right"
              )

fb_mean = round(res['fb_mape_OOS'].mean(), 3)
fig.add_hline(y=fb_mean, line_dash="dot", line_color='blue',
              annotation_text=str(fb_mean),
              annotation_position="top left")

fb_withPromo_mean = round(res['fb_withPromo_mape_OOS'].mean(), 3)
fig.add_hline(y=fb_withPromo_mean, line_dash="dot", line_color='red',
              annotation_text=str(fb_withPromo_mean),
              annotation_position="top right")
